In [1]:
# !pip install python-dotenv
# !pip install requests

## Imports

In [2]:
# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# For enums
from enum import StrEnum

# To convert to json objects
import json

# For Http calls
import requests

# For dataframe
import pandas as pd

# Date conversions
import time

## Load environment variables

In [3]:
dotenv_path = Path('.env/graph_alchemy')
load_dotenv(dotenv_path=dotenv_path)
GRAPHQL_ENDPOINT = os.getenv('GRAPHQL_ENDPOINT')
WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')

## Constants

In [4]:
# Pool token symbol pair
SYMBOL0 = 'USDC'
SYMBOL1 = 'WETH'

# Fee tier (0.05%)
FEE_TIER = 500

# Timeout in seconds
TIMEOUT = 40

# Actions
class ActionType(StrEnum):
    DEPOSIT = 'Deposit'
    COLLECT = 'Collect Fee'
    WITHDRAW = 'Withdraw'

# Query to get the pool id
POOL_QUERY = """
{{
  pools(
    where: {{token0_: {{symbol: "{symbol0}"}},
    token1_: {{symbol: "{symbol1}"}}, feeTier: "{fee_tier}"}}
  ) {{
    id
  }}
}}
"""

# Query to get Mint positions
MINT_QUERY =  """
{{
  mints(
    where: {{
      pool: "{pool}",
      origin: "{owner}"
      amountUSD_gt: 0
      timestamp_gte: {start}
    }}
    orderBy: timestamp
  ) {{
        id
        timestamp
        amount0
        amount1
        amountUSD
  }}
}}
"""

# Query to get burn posiitons
BURN_QUERY = """
{{
  burns(
    where: {{
      pool: "{pool}",
      origin: "{owner}"
      amountUSD_gt: 0
      timestamp_gte: {start}
    }}
    orderBy: timestamp
  ) {{
        id
        timestamp
        amount0
        amount1
        amountUSD
    }}
}}
"""

# Collect query to get the USD amoutn for collection
COLLECT_QUERY = """
{{
  collects(
    where: {{
      pool: "{pool}"
      transaction: "{tx}"
    }}
  ) {{
        id
        timestamp
        amount0
        amount1
        amountUSD
    }}
}}

"""
# Directory to read transactions
TRANS_DIR = 'lp_transactions'

In [5]:
def get_pool_id(symbol0:str, symbol1:str, fee_tier:int) -> str:
    """ Returns the pool id
    Parameters:
    symbol0 : str
        token0 symbol
    symbol1 : str
        token1 symbol
    fee_tier: int
        fee tier
        
    Returns:
    str
        pool id
    """   
    query = POOL_QUERY.format(symbol0=symbol0, symbol1=symbol1, fee_tier=fee_tier)
    json_data = json.loads(requests.post(url=GRAPHQL_ENDPOINT, json={'query': query}, timeout=TIMEOUT).text)
    # Extract the pool id
    return json_data['data']['pools'][0]['id']

In [6]:
pool_id = get_pool_id(symbol0=SYMBOL0, symbol1=SYMBOL1, fee_tier=FEE_TIER)
pool_id

'0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'

In [7]:
def load_transactions(pool_id:str) -> pd.DataFrame:
    """ Loads transaction from a file
    Parameters:
    pool_id : str
        pool id to construct the filename
    Returns: pd.DataFrame
    A dataframe with relevant transactions
    """
    filename = '{path}{sep}{part1}-{part2}.json'.\
        format(path=TRANS_DIR, sep=os.sep, part1=WALLET_ADDRESS[:6], part2=pool_id[:6])
    
    # Read transaction from file
    with open(filename, 'r') as read_file:
        data = json.load(read_file)
    # Create a DF from json file
    df = pd.DataFrame(data)
    # Only select rows with either amount0 or amount1 has a value
    df = df.query('(amount0 > 0 or amount1 > 0)')
    # Reset amount columns we are getting values from graph (you can keep them as they are as well)
    df[['amount0','amount1']] = 0.0
    # Add a USD column which we will populate from GraphQL
    df['amountUSD'] = 0.0
    # Add enum type to action column
    df['action'] = df['action'].apply(lambda x: ActionType(x))
    # df.rename(columns={'A': 'Col_1', 'C': 'Col_3'}))
    df.sort_values(by=['token_id','time_stamp', 'log_index'], inplace=True)
    return df

In [8]:
df = load_transactions(pool_id=pool_id)
df.head()

,token_id,time_stamp,hash,log_index,action,amount0,amount1,amountUSD
18,903752,1736400443,0x1ad2cc022af3e37ce15ae5055e8ee24e4bbf226a3b99...,218,Deposit,0.0,0.0,0.0
6,903752,1736553551,0xd0b32d113cb235e59b48c4021fcd19ba1cb31c745109...,173,Collect Fee,0.0,0.0,0.0
40,903752,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,31,Withdraw,0.0,0.0,0.0
41,903752,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,35,Collect Fee,0.0,0.0,0.0
42,907493,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,58,Deposit,0.0,0.0,0.0


In [9]:
def update_amounts(df:pd.DataFrame, data:list[str], action:ActionType) -> None:
    """ Updates amount0, amount1 and amountUSD
    Parameters:
    df : pd.DataFrame
        DF containing data to update        
    data : list[str]
        An array items to loop
    action: ActionType
        Action type
    Returns:
    None
    """
    # Loop through mints/burns/collects
    for item in data:
        # Extract tx hash from id
        tx_hash, _ = item['id'].split('-')
        # Save the condition
        mask = (df['hash'] == tx_hash) & (df['time_stamp'] == int(item['timestamp'])) \
            & (df['action'] == action)
        # Update amounts and USD for the matching record
        for col in ['amount0', 'amount1', 'amountUSD']:
            df.loc[mask, col] = float(item[col])

## Update Deposits/Withdrawls USD

In [10]:
def update_deposit_and_withdraw(action:ActionType, df:pd.DataFrame) -> None:
    """ Updates Deposit/Withdrawal actions with USD
    Parameters:
    action: ActionType
        Action types
    df : pd.DataFrame
        DF containing data to update        
    Returns:
    None
    """   
    # Start TS for mint search
    first_mint_ts = df[df['action'] == action]['time_stamp'].iloc[0].item()
    # Construct the query
    if action == ActionType.DEPOSIT:
        query = MINT_QUERY.format(pool=pool_id, owner=WALLET_ADDRESS, start= first_mint_ts)
        label = 'mints'
    else:
        query = BURN_QUERY.format(pool=pool_id, owner=WALLET_ADDRESS, start= first_mint_ts)
        label = 'burns'
    json_data = json.loads(requests.post(url=GRAPHQL_ENDPOINT, json={'query': query}, timeout=TIMEOUT).text)

    # Loop through mints/burns
    update_amounts(df=df, data=json_data['data'][label], action=action)

In [11]:
# Update Deposits and Withdraws
update_deposit_and_withdraw(action=ActionType.DEPOSIT, df=df)
update_deposit_and_withdraw(action=ActionType.WITHDRAW, df=df)
df.head()

,token_id,time_stamp,hash,log_index,action,amount0,amount1,amountUSD
18,903752,1736400443,0x1ad2cc022af3e37ce15ae5055e8ee24e4bbf226a3b99...,218,Deposit,6561.830118,2.030305,13325.037428
6,903752,1736553551,0xd0b32d113cb235e59b48c4021fcd19ba1cb31c745109...,173,Collect Fee,0.000000,0.000000,0.000000
40,903752,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,31,Withdraw,919.460129,3.756619,12911.491795
41,903752,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,35,Collect Fee,0.000000,0.000000,0.000000
42,907493,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,58,Deposit,6366.530436,2.072485,12982.401368


In [12]:
def update_collect(df:pd.DataFrame) -> None:
    """ Updates Collect Fee actions with USD
    Parameters:
    df : pd.DataFrame
        DF containing data to update        
    Returns:
    None
    """
    # Select 'Collect Fee' action items
    collect_fee_ser = df[df['action'] == ActionType.COLLECT]['hash']
    for indx in collect_fee_ser:
        # Need to call with each transaction 
        query = COLLECT_QUERY.format(pool=pool_id, tx=indx)
        json_data = json.loads(requests.post(url=GRAPHQL_ENDPOINT, json={'query': query}, timeout=TIMEOUT).text)
        # Update amounts        
        update_amounts(df=df, data=json_data['data']['collects'], action=ActionType.COLLECT)

In [13]:
update_collect(df)
df.head()

,token_id,time_stamp,hash,log_index,action,amount0,amount1,amountUSD
18,903752,1736400443,0x1ad2cc022af3e37ce15ae5055e8ee24e4bbf226a3b99...,218,Deposit,6561.830118,2.030305,13325.037428
6,903752,1736553551,0xd0b32d113cb235e59b48c4021fcd19ba1cb31c745109...,173,Collect Fee,83.241689,0.026011,168.317629
40,903752,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,31,Withdraw,919.460129,3.756619,12911.491795
41,903752,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,35,Collect Fee,955.841644,3.768208,12984.871263
42,907493,1736753255,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,58,Deposit,6366.530436,2.072485,12982.401368


In [14]:
# token_id, time_stamp, tx_hash, amount0, amount1, amountUSD = df_dup.iloc[0][
#     ['token_id','time_stamp','hash','amount0','amount1','amountUSD']]
# mask = (df['token_id'] == token_id) & (df['time_stamp'] == time_stamp) & \
#     (df['hash'] == tx_hash) & (df['action'] == 'Withdraw1')
# _, _, _, amount_0, amount_1, amount_USD = df.loc[mask, ['token_id','time_stamp','hash','amount0','amount1','amountUSD']].iloc[0]
# amount_0.item()

In [15]:
def adjust_collect_amounts(df:pd.DataFrame) -> None:
    """ Adjust Collect Fees to reflect fees
    Parameters:
    df : pd.DataFrame
        DF containing data to update        
    Returns:
    None
    """
    # Get duplicate records
    df_dup = df[df.duplicated(subset=['token_id','time_stamp', 'hash'])]
    for row in df_dup.itertuples():
        if row.action == ActionType.COLLECT:
            mask = (df['token_id'] == row.token_id) & (df['time_stamp'] == row.time_stamp) & \
                (df['hash'] == row.hash) & (df['action'] == ActionType.WITHDRAW)
            # Get the Withdraw amounts and subtract them from Collect amounts
            _, _, _, amount_0, amount_1, amount_USD = df.loc[mask, 
                ['token_id','time_stamp','hash','amount0','amount1','amountUSD']].iloc[0]
            fee_amount0 = row.amount0 - amount_0
            fee_amount1 = row.amount1 - amount_1
            fee_amountUSD = row.amountUSD - amount_USD
            # Mask for the Collect record
            mask = (df['token_id'] == row.token_id) & (df['time_stamp'] == row.time_stamp) & \
                (df['hash'] == row.hash) & (df['action'] == ActionType.COLLECT)
        else:
            # Mask for the Collect as the current row is the Withdraw
            mask = (df['token_id'] == row.token_id) & (df['time_stamp'] == row.time_stamp) & \
                (df['hash'] == row.hash) & (df['action'] == ActionType.COLLECT)
            # Subtract Collect amounts from Withdraw amounts
            _, _, _, amount_0, amount_1, amount_USD = df.loc[mask, 
                ['token_id','time_stamp','hash','amount0','amount1','amountUSD']].iloc[0]
            fee_amount0 = amount_0 - row.amount0
            fee_amount1 = amount_1 - row.amount1
            fee_amountUSD = amount_USD - row.amountUSD
        # Update the Collect record with adjusted amounts
        df.loc[mask, ['amount0', 'amount1', 'amountUSD']] = fee_amount0, fee_amount1, fee_amountUSD    

In [16]:
# Adjust fees
adjust_collect_amounts(df)

# Rename headings and convert timestamp
df.columns = ['TokenId', 'Timestamp', 'Hash', 'LogIndex', 'Action', SYMBOL0, SYMBOL1, 'USD']
df['Timestamp'] = df['Timestamp'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x)))
df.head()

,TokenId,Timestamp,Hash,LogIndex,Action,USDC,WETH,USD
18,903752,2025-01-09 05:27:23,0x1ad2cc022af3e37ce15ae5055e8ee24e4bbf226a3b99...,218,Deposit,6561.830118,2.030305,13325.037428
6,903752,2025-01-10 23:59:11,0xd0b32d113cb235e59b48c4021fcd19ba1cb31c745109...,173,Collect Fee,83.241689,0.026011,168.317629
40,903752,2025-01-13 07:27:35,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,31,Withdraw,919.460129,3.756619,12911.491795
41,903752,2025-01-13 07:27:35,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,35,Collect Fee,36.381515,0.011590,73.379468
42,907493,2025-01-13 07:27:35,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,58,Deposit,6366.530436,2.072485,12982.401368


## Utility methods for styling

In [17]:
def style_action(column):    
    deposit_style = 'color: firebrick;'
    default = 'color: forestgreen'

    # must return one string per cell in this column
    return [deposit_style if v == 'Deposit' else default for v in column]

def make_clickable(val):
    url = 'https://etherscan.io/tx/{hash}'.format(hash=val)
    link_name = '{first_part} ... {last_part}'.format(first_part=val[:8], last_part=val[-4:])
    return f'<a target="_blank" href="{url}">{link_name}</a>'

## Apply styles to DF

In [18]:
# Table title
table_caption = 'LP events for the wallet {first_part} ... \
    {last_part} - (Uniswap Pool {symbol0}/{symbol1} {fee}%)'.\
    format(first_part=WALLET_ADDRESS[:4], last_part=WALLET_ADDRESS[-4:],
           symbol0=SYMBOL0, symbol1=SYMBOL1, fee=FEE_TIER/10000)

# Apply styles
df.style.\
    hide(axis='index').\
    set_caption(table_caption).\
    set_properties(**{'border': '0.1px solid black'}).\
    set_properties(subset=['Action'], **{'text-align': 'left'}).\
    set_table_styles([
        {'selector': 'th.col_heading', 'props': 'text-align: center'},
        {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '12pt')]}]).\
    apply(style_action, subset=['Action'], axis=0).\
    format({'Hash': make_clickable, SYMBOL0:'{:,.2f}', 'USD':'{:,.2f}'})

TokenId,Timestamp,Hash,LogIndex,Action,USDC,WETH,USD
903752,2025-01-09 05:27:23,0x1ad2cc ... b917,218,Deposit,"6,561.83",2.030305,"13,325.04"
903752,2025-01-10 23:59:11,0xd0b32d ... 8f52,173,Collect Fee,83.24,0.026011,168.32
903752,2025-01-13 07:27:35,0xb9f1bc ... e4e3,31,Withdraw,919.46,3.756619,"12,911.49"
903752,2025-01-13 07:27:35,0xb9f1bc ... e4e3,35,Collect Fee,36.38,0.011590,73.38
907493,2025-01-13 07:27:35,0xb9f1bc ... e4e3,58,Deposit,"6,366.53",2.072485,"12,982.40"
907493,2025-01-16 08:46:23,0x38e94c ... f1f8,408,Collect Fee,115.14,0.035420,233.44
907493,2025-01-16 09:13:11,0x2a00b9 ... 45fa,203,Withdraw,"10,716.83",0.735192,"13,150.44"
907493,2025-01-16 09:13:11,0x2a00b9 ... 45fa,207,Collect Fee,0.85,0.000403,2.18
909702,2025-01-16 09:13:11,0x2a00b9 ... 45fa,227,Deposit,"6,206.01",2.095518,"13,142.52"
909702,2025-01-18 21:06:11,0x2866ea ... bd9b,200,Collect Fee,73.90,0.022255,146.57
